In [25]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [26]:
#search page
browser = Browser('chrome')
url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=pdr&search_distance=4#search=1~gallery~0~0'
browser.visit(url)
html = browser.html
website = soup(html, 'html.parser')

In [29]:
#collect all of the search results

results = website.find_all(class_="cl-search-result cl-search-view-mode-gallery")

results_info = []

for x in results:
    if x.find(class_='titlestring') is not None:
        title = x.find(class_='titlestring').text
        link = x.find(class_='titlestring')['href']
    else: title = None
    if x.find(class_='meta') is not None:
        info = x.find(class_='meta').text
    else: info = None
    if x.find(class_='priceinfo') is not None:
        price = x.find(class_='priceinfo').text
    else: price = None
    table = {
        "title":title,
        "link":link,
        "info":info,
        "price":price,
    }
    results_info.append(table)

In [32]:
# df = pd.DataFrame(results_info)
# df
# table

NameError: name 'table' is not defined

In [5]:
meta = website.find("div", class_='meta')
meta
# meta = meta.find("span", class_="separator").remove()

In [8]:
#with many more results

url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=industrial&search_distance=4#search=1~gallery~0~0'
browser.visit(new_url)
html = browser.html
website = soup(html, 'html.parser')

results = website.find_all(class_="cl-search-result cl-search-view-mode-gallery")

results_info = []

for x in many_results:
    if x.find(class_='titlestring') is not None:
        title = x.find(class_='titlestring').text
        link = x.find(class_='titlestring')['href']
    else: new_title = None
    if x.find(class_='meta') is not None:
        info = x.find(class_='meta').text
    else: info = None
    if x.find(class_='priceinfo') is not None:
        price = x.find(class_='priceinfo').text
    else: price = None
    dictionary = {
        "title":title,
        "link":link,
        "info":info,
        "price":price,
    }
    results_info.append(dictionary)

In [10]:
big_df = pd.DataFrame(results_info)
# big_df["info"] = big_df["info"].replace("·","/")
# big_df[["date", "sqft","neighborhood"]] = big_df["info"].str.split("·",expand=True)
big_df.head()

""


In [12]:
#across multiple pages

url = 'https://sfbay.craigslist.org/search/off?postal=94107&query=industrial&search_distance=4#search=1~gallery~0~0'
browser.visit(url)

for x in range(1,4):
    html = browser.html
    website = soup(html, 'html.parser')
    results = website.find_all(class_="cl-search-result cl-search-view-mode-gallery")

    all_results_info = []

    for x in results:
        if x.find(class_='titlestring') is not None:
            title = x.find(class_='titlestring').text
            link = x.find(class_='titlestring')['href']
        else: title = None
        if x.find(class_='meta') is not None:
            info = x.find(class_='meta').text
        else: info = None
        if x.find(class_='priceinfo') is not None:
            price = x.find(class_='priceinfo').text
        else: price = None
#         browser.find_by_css('button[class_="bd-button cl-next-page icon-only"]').click()
        dictionary = {
            "title":title,
            "link":link,
            "info":info,
            "price":price,
        }
        results_info.append(dictionary)

In [23]:
browser.quit()